# Squirro machine learning service walkthrough

## Settings

In [90]:
CLUSTER='https://unstable.squirro.net'
TOKEN='5de370f686a7bf4376eaab5080670c932fd9dd7bb3e2f79ef5d345571b992ad3f4b067aee6fd4f7990a08d009bb374135a7345b31f215892d95b08c515f6118f'
PROJECT_ID='gqunGq56Qw6bIX36GLy6hg'

## Get Squirro client

In [91]:
from squirro_client import SquirroClient
client = SquirroClient(client_id=None, client_secret=None, cluster=CLUSTER)
client.authenticate(refresh_token=TOKEN)

## Create machine learning workflow

The workflow defines:
    - what data you'd like to train and infer on
    - the keywords you'll use as features and labels
    - what normalization/filtering/tokenization steps are required to manipulate the text
    - what models you want to use and their associated hyperparameters

In [92]:
ml_workflow = {
  "dataset": {
    "train": {"query_string": "dataset:train (label:sci.space OR label:soc.religion.christian OR label:alt.atheism)"},
    "process": {"query_string": "dataset:test (label:sci.space OR label:soc.religion.christian OR label:alt.atheism)"}
  },
  "analyzer": {
    "type": "classification",
    "tag_field": "keywords.pred_label",
    "label_field": "keywords.label"
  },
  "pipeline": [{
    "step": "loader",
    "type": "squirro_query",
    "fields": ["body", "title", "keywords.label"]
  },{
    "step": "filter",
    "type": "empty",
    "fields": ["body", "title", "keywords.label"]
  },{
    "step": "filter",
    "type": "join",
    "input_field": "keywords.label",
    "output_field": "keywords.label"
  },{
    "step": "filter",
    "type": "merge",
    "input_fields": ["body", "title"],
    "output_field": "text"
  },{
    "step": "normalizers",
    "types": ["html", "punctuation", "lowercase", "character"],
    "fields": ["text"]
  },{
    "step": "tokenizer",
    "type": "spaces",
    "fields": ["text"]
  },{
    "step": "embedder",
    "type": "dictionary",
    "batch_size": 1024,
    "input_field": "text",
    "output_field": "indexed_text"
  },{
    "step": "checkpoint",
    "type": "disk",
    "do_randomize": True,
    "batch_size": 1
  },{
    "step": "classifier",
    "type": "cnn_seq2one",
    "batch_size": 1024,
    "dict_name": "dictionary",
    "dropout_fraction": 0.5,
    "embedding_dim": 50,
    "explanation_field": "explanantion",
    "input_field": "indexed_text",
    "label_field": "keywords.label",
    "labels": ["soc.religion.christian", "alt.atheism", "sci.space"],
    "max_sequence_length": 1000,
    "mini_batch_size": 64,
    "n_epochs": 10,
    "output_field": "keywords.pred_label"
  },{
    "step": "saver",
    "type": "squirro_item",
    "batch_size": 1000,
    "fields": ["keywords.pred_label"]
  }]
}

## Upload workflow

This hands the workflow configuration (and any local pre-trained models) to the Squirro ML service.

In [93]:
print len(client.get_machinelearning_workflows(PROJECT_ID).get('machinelearning_workflows'))

0


In [94]:
ml_workflow_id = client.new_machinelearning_workflow(
    PROJECT_ID, name='e2e_cnn', config=ml_workflow).get('id')

## Create training job

We now tell Squirro that we want a training job for the ML workflow we just uploaded. This will train the models we defined. If nothing is in the queue, it should start immediately.

In [95]:
print len(client.get_machinelearning_jobs(
    PROJECT_ID, ml_workflow_id=ml_workflow_id).get('machinelearning_jobs'))

0


In [96]:
training_job_id = client.new_machinelearning_job(
    PROJECT_ID, ml_workflow_id=ml_workflow_id, type='training').get('id')

In [97]:
import time
def wait_for_ml_job(project_id, ml_workflow_id, ml_job_id):
    """Wait for ML job to finish"""
    while True:
        job = client.get_machinelearning_job(
            project_id, ml_workflow_id, ml_job_id).get('machinelearning_job')
        if job.get('last_error_at') is not None or job.get('last_success_at') is not None:
            print job
            break
        time.sleep(1)
    return job

In [98]:
print wait_for_ml_job(PROJECT_ID, ml_workflow_id, training_job_id)

{u'total_runs': 1, u'next_run_time_at': u'2018-05-25T02:21:55', u'healthy': True, u'last_error': None, u'created_at': u'2018-05-24T08:08:57', u'modified_at': u'2018-05-24T08:18:01', u'last_error_at': None, u'ml_workflow_id': u'esr-srLARBmN47dAweaifg', u'last_success_at': u'2018-05-24T08:18:01', u'type': u'training', u'id': u'ks9GZEZCTuqcvIjiBE7Nxw', u'error_count': 0}
{u'total_runs': 1, u'next_run_time_at': u'2018-05-25T02:21:55', u'healthy': True, u'last_error': None, u'created_at': u'2018-05-24T08:08:57', u'modified_at': u'2018-05-24T08:18:01', u'last_error_at': None, u'ml_workflow_id': u'esr-srLARBmN47dAweaifg', u'last_success_at': u'2018-05-24T08:18:01', u'type': u'training', u'id': u'ks9GZEZCTuqcvIjiBE7Nxw', u'error_count': 0}


## Test on unlabeled data

Just to make sure the model training is succesful, we try a synchronous inference on a few test items. This `run_machinelearning_workflow` is the same command used in the ML service pipelet: https://github.com/squirro/delivery/tree/master/templates/pipelets/machinelearning.

In [99]:
items = client.query(
    PROJECT_ID,
    query="dataset:test (label:sci.space OR label:soc.religion.christian OR label:alt.atheism)",
    fields=["body", "title", "keywords"],
    count=3
).get('items')
print(len(items))

3


In [100]:
items = client.run_machinelearning_workflow(
    PROJECT_ID, ml_workflow_id=ml_workflow_id, data={'items': items}).get('items')
for item in items:
    print item.get('keywords').get('label'), item.get('keywords').get('pred_label')

[u'soc.religion.christian'] [{u'soc.religion.christian': 0.9988566637}, {u'sci.space': 0.0}, {u'alt.atheism': 0.0011433086}]
[u'soc.religion.christian'] [{u'soc.religion.christian': 1.0}, {u'sci.space': 0.0}, {u'alt.atheism': 2.14e-08}]
[u'soc.religion.christian'] [{u'soc.religion.christian': 0.0249485839}, {u'sci.space': 0.000496073}, {u'alt.atheism': 0.9745553732}]


## Create inference job

For regularly changing datasets, it is advantageous to set up scheduled inference jobs. These will run asynchronously to free up ingestion. Again, if nothing is in the queue, these should run immediately.

In [101]:
print len(client.get_machinelearning_jobs(PROJECT_ID, ml_workflow_id=ml_workflow_id))

1


In [102]:
inference_job_id = client.new_machinelearning_job(
    PROJECT_ID, ml_workflow_id=ml_workflow_id, type='inference').get('id')

In [103]:
print wait_for_ml_job(PROJECT_ID, ml_workflow_id, inference_job_id)

{u'total_runs': 1, u'next_run_time_at': u'2018-05-25T12:26:49', u'healthy': True, u'last_error': None, u'created_at': u'2018-05-24T08:18:06', u'modified_at': u'2018-05-24T08:18:12', u'last_error_at': None, u'ml_workflow_id': u'esr-srLARBmN47dAweaifg', u'last_success_at': u'2018-05-24T08:18:12', u'type': u'inference', u'id': u'vTO5K0ecQNyUU4fWCrhhdQ', u'error_count': 0}
{u'total_runs': 1, u'next_run_time_at': u'2018-05-25T12:26:49', u'healthy': True, u'last_error': None, u'created_at': u'2018-05-24T08:18:06', u'modified_at': u'2018-05-24T08:18:12', u'last_error_at': None, u'ml_workflow_id': u'esr-srLARBmN47dAweaifg', u'last_success_at': u'2018-05-24T08:18:12', u'type': u'inference', u'id': u'vTO5K0ecQNyUU4fWCrhhdQ', u'error_count': 0}


## Analyze training

When training any model, it is important to iteratively check how well your models are doing. In the future, this functionality will be largely in the GUI. For now, however, we can check via libNLP (see libNLP walkthrough).

## reset

In [104]:
for ml_workflow in client.get_machinelearning_workflows(PROJECT_ID).get('machinelearning_workflows'):
    client.delete_machinelearning_workflow(PROJECT_ID, ml_workflow_id=ml_workflow.get('id'))

# Gotchyas

- multi-node training jobs not currently working
- underfitting
- overfitting
- hyperparameter tuning
- make sure documents make it through the pipeline
- know your baselines